In [29]:
import math

def profit(x):
    x = [round(val) for val in x]  # convert x into integer
    s = sales(x)  # get the expected sales
    c = cost(s)  # get the expected cost
    profit_val = sum([val[0] * val[1] - val[2] for val in zip(s, x, c)])  # compute the profit
    return profit_val

def cost(units, A=100, cpu=lambda units: [35 - 5 * i for i in range(1, len(units) + 1)]):
    return [A + cpu_val * unit for cpu_val, unit in zip(cpu(units), units)]

def sales(x, A=1000, B=200, C=141, m=None):
    if m is None:
        m = [2 - 0.25 * i for i in range(1, len(x) + 1)]
    return [round(marketing * (A / math.log(price + B) - C)) for price, marketing in zip(x, m)]

def fact(x):
    if x == 0:
        return 1
    else:
        return x * fact(x - 1)
#===========================================================================================

# Test the functions:
x = [414.1, 404.2, 408.3, 413.2, 395.0]
print("class of profit is:", type(profit))  # function

y = profit(x)
print("maximum profit:", y)

print("x is not changed:", x)

print("cost(x=", x, ")=", cost(x))
print("sales(x=", x, ")=", sales([round(val) for val in x]))
x = [414, 404]  # sales for 2 bags
print("sales(x=", x, ")=", sales(x))
print("sales(x, A=1000)=", sales(x, 1000))
x = 3
print("fact(", x, ")=", fact(x))

class of profit is: <class 'function'>
maximum profit: 36163
x is not changed: [414.1, 404.2, 408.3, 413.2, 395.0]
cost(x= [414.1, 404.2, 408.3, 413.2, 395.0] )= [12523.0, 10205.0, 8266.0, 6298.0, 4050.0]
sales(x= [414.1, 404.2, 408.3, 413.2, 395.0] )= [26, 23, 19, 15, 12]
sales(x= [414, 404] )= [26, 23]
sales(x, A=1000)= [26, 23]
fact( 3 )= 6


In [20]:
import numpy as np

def fsearch(search, fn, type="min", *args):
    x = np.apply_along_axis(fn, 1, search, *args)
    ib = np.argmin(x) if type == "min" else np.argmax(x)
    return {"index": ib, "sol": search[ib], "eval": x[ib]}

def dfsearch(l=1, b=1, domain=None, fn=None, type="min", par=None, bcur=None, Dim=None, *args):
    D = len(domain) if par is None else len(par)

    if par is None:
        par = np.array([None] * D)

    if bcur is None:
        bcur = {"sol": None, "eval": float("inf") if type == "min" else float("-inf")}

    if l - 1 == D:
        f = fn(par, Dim, *args) if Dim is not None else fn(par, *args)
        fb = bcur["eval"]
        ib = np.argmin([fb, f]) if type == "min" else np.argmax([fb, f])
        if ib == 0:
            return bcur
        else:
            return {"sol": par.tolist(), "eval": f}
    else:
        for j in range(len(domain[l - 1])):
            par[l - 1] = domain[l - 1][j]
            bcur = dfsearch(l + 1, j + 1, domain, fn, type, par.copy(), bcur, Dim, *args)
        return bcur

def binint(x, D):
    x = list(reversed(format(x, 'b').zfill(D)))
    return [int(bit) for bit in x]

def intbin(x):
    return sum([2**i for i, bit in enumerate(reversed(x)) if bit == 1])

def sumbin(x):
    return sum(x)

def maxsin(x, Dim):
    return np.sin(np.pi * intbin(x) / (2 ** Dim))

D = 8
x = np.arange(2 ** D)
search = np.array([binint(val, D) for val in x])
domain = [[0, 1] for _ in range(D)]

S1 = fsearch(search, sumbin, "max")
print("fsearch best s:", S1["sol"], "f:", S1["eval"])

S2 = dfsearch(domain=domain, fn=sumbin, type="max")
print("dfsearch best s:", S2["sol"], "f:", S2["eval"])

Dim = search.shape[1]
S3 = fsearch(search, maxsin, "max", Dim)
print("fsearch best s:", S3["sol"], "f:", S3["eval"])

S4 = dfsearch(domain=domain, fn=maxsin, type="max", Dim=Dim)
print("dfsearch best s:", S4["sol"], "f:", S4["eval"])

fsearch best s: [1 1 1 1 1 1 1 1] f: 8
dfsearch best s: [1, 1, 1, 1, 1, 1, 1, 1] f: 8
fsearch best s: [1 0 0 0 0 0 0 0] f: 1.0
dfsearch best s: [1, 0, 0, 0, 0, 0, 0, 0] f: 1.0


In [30]:
def ibag(D):
    x = np.arange(1, 1001)  # price for each bag type
    # set search space for one bag:
    search = np.ones((1000, 5))
    search[:, D - 1] = x
    S1 = fsearch(search, profit, "max")
    return S1['sol'][D - 1]  # best price

# Compute the best price for all bag types:
S = [ibag(D) for D in range(1, 6)]
# Show the optimum solution:
print("optimum s:", S, "f:", profit(S))

optimum s: [404.0, 408.0, 413.0, 395.0, 399.0] f: 36622
